<a href="https://colab.research.google.com/github/Rafaelfyh/CliNER/blob/master/Modeling_Hackaton_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import statsmodels.api as sm
import patsy as pt
from google.colab import drive
from sklearn import metrics
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

In [ ]:
df1 = pd.read_csv('/content/features.csv')

In [ ]:
df2 = pd.read_csv('/content/stores.csv')

In [ ]:
df3 = pd.read_csv('/content/train.csv')

In [ ]:
dft =df1.merge(df3, on = ['Store','Date','IsHoliday'])

In [ ]:
dft = dft.merge(df2, on = ['Store'])

In [ ]:
dft['Date']=pd.to_datetime(dft['Date'])

In [ ]:
dft['IsHoliday'] = dft['IsHoliday'].apply(lambda x: 0 if x ==False else 1)

In [ ]:
type_colunas = pd.get_dummies(dft['Type'])
for i in type_colunas.columns:
  dft['Type_' + i] = type_colunas[i]
dft.drop('Type',axis=1,inplace=True) 

In [ ]:
loja_colunas = pd.get_dummies(dft['Store'].astype(str))
for i in loja_colunas.columns:
  dft['Store_' + i] = loja_colunas[i]
dft.drop('Store',axis=1,inplace=True)

In [ ]:
dept_colunas = pd.get_dummies(dft['Dept'].astype(str))
for i in dept_colunas.columns:
  dft['Dept_' + i] = dept_colunas[i]
dft.drop('Dept',axis=1,inplace=True)

In [ ]:
dft=dft.fillna(0)

In [ ]:
dft.shape

(421570, 142)

In [ ]:
dfsample = dft.sample(10000)

In [ ]:
X = dfsample.drop(['Weekly_Sales','Date'], axis=1)
y = dfsample['Weekly_Sales']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
y_train.shape

(7000,)

In [ ]:
colunas = ['Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3',
                 'MarkDown4','MarkDown5','CPI','Unemployment','Size'] 

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[colunas])

StandardScaler()

scaler sempre retornara um dicionario

normalização de colunas não boleanas

In [ ]:
X_train = X_train.reset_index().drop('index',axis=1)
X_test = X_test.reset_index().drop('index',axis=1)
y_train = y_train.reset_index().drop('index',axis=1)
y_test = y_test.reset_index().drop('index',axis=1)

X_train[colunas] = pd.DataFrame(scaler.transform(X_train[colunas]),columns=colunas)

X_test[colunas] = pd.DataFrame(scaler.transform(X_train[colunas]),columns=colunas)
 

In [ ]:
X_train.head()

,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,...,Dept_90,Dept_91,Dept_92,Dept_93,Dept_94,Dept_95,Dept_96,Dept_97,Dept_98,Dept_99
0,1.187916,0.926099,-0.428985,-0.167231,-0.086334,-0.281618,-0.420086,1.101447,0.176471,0,...,0,0,0,0,0,0,0,0,0,0
1,1.099664,0.562312,0.664166,-0.150194,-0.086329,0.881909,3.121347,1.269681,-0.717386,0,...,0,0,0,0,0,0,0,1,0,0
2,0.318010,0.976503,-0.428985,-0.167231,-0.086334,-0.281618,-0.420086,-0.881337,0.490946,0,...,0,0,0,0,0,0,0,0,0,0
3,0.683623,0.312483,0.553349,-0.146419,-0.053414,-0.131138,0.451484,1.301765,-0.927924,0,...,0,0,0,0,0,0,0,0,0,0
4,0.827237,0.711333,1.012755,-0.167231,-0.084607,0.763296,0.480839,1.258355,-0.522304,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV, HuberRegressor, LassoLars, BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn import metrics

In [ ]:
modelos = {'svr':SVR(),
           'dtr':DecisionTreeRegressor()
           }

In [ ]:
models = {
    'LinearReg': LinearRegression(),
    'RidgeReg': Ridge(),
    'LassoReg': Lasso(),
    'ElasticNetReg': ElasticNet(),
    'HuberReg': HuberRegressor(),
    'LassoCV': LassoCV(),
    'ElasticNetCV': ElasticNetCV(),
    'LassoLars': LassoLars(),
    'BayesianRidge': BayesianRidge(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'SVM_RBF':SVR(kernel='rbf'),
    'SVM_POLY':SVR(kernel='poly'),
    'SVM_LINEAR':SVR(kernel='linear',max_iter=10**4),
    'SVM_SIG':SVR(kernel='sigmoid'),
    'SGDRegressor':SGDRegressor(),
    'RandomForestRegressor':RandomForestRegressor(),
    'AdaBoostRegressor':AdaBoostRegressor(),
    'GradientBoostingRegressor':GradientBoostingRegressor(),
    'ExtraTreesRegressor':ExtraTreesRegressor()
} 

In [ ]:
for model in models.values():
   model.fit(X_train, y_train)

eval = []
nome = []
r2 = []
MAE = []
for name, model in models.items():
    y_pred = model.predict(X_test)
    nome.append(name)
    eval.append(metrics.mean_squared_error(y_test,y_pred))
    r2.append(metrics.r2_score(y_test,y_pred))
    MAE.append(metrics.mean_absolute_error(y_test,y_pred))
    print('-------------------------')
    print(name + ":\n R² : {:.4f}\n MSE: {:.4F}\n MAE: {:.4F}".format(metrics.r2_score(y_test,y_pred),metrics.mean_squared_error(y_test,y_pred),metrics.mean_absolute_error(y_test,y_pred)))
    print('-------------------------') 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate

-------------------------
LinearReg:
 R² : -29250952606128132849664.0000
 MSE: 16572818329724448885073845944320.0000
 MAE: 3723652116901613.5000
-------------------------
-------------------------
RidgeReg:
 R² : 0.4422
 MSE: 316023418.0812
 MAE: 11341.7302
-------------------------
-------------------------
LassoReg:
 R² : 0.5324
 MSE: 264919692.1497
 MAE: 9328.7774
-------------------------
-------------------------
ElasticNetReg:
 R² : 0.0019
 MSE: 565485592.4272
 MAE: 13882.4615
-------------------------
-------------------------
HuberReg:
 R² : 0.4119
 MSE: 333227787.2923
 MAE: 11232.8965
-------------------------
-------------------------
LassoCV:
 R² : 0.5332
 MSE: 264456830.3830
 MAE: 9197.2735
-------------------------
-------------------------
ElasticNetCV:
 R² : -0.0023
 MSE: 567879087.9164
 MAE: 14847.2183
-------------------------
-------------------------
LassoLars:
 R² : 0.3385
 MSE: 374814949.4924
 MAE: 13617.4956
-------------------------
-------------------------
Baye

In [ ]:

reg=LassoCV()
reg.fit(X_train, y_train)







In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Otimização de hiperparametro
LassoCVdicio={'n_alphas':[50,100,150],
'max_iter':[500, 1000, 2000]}

clf = GridSearchCV(reg, LassoCVdicio)
clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expec

GridSearchCV(estimator=LassoCV(),
             param_grid={'max_iter': [500, 1000, 2000],
                         'n_alphas': [50, 100, 150]})

In [ ]:
y_pred=clf.predict(X_test)

In [ ]:
print(metrics.mean_absolute_error(y_test,y_pred))

print(metrics.r2_score(y_test,y_pred))

print(metrics.mean_squared_error(y_test,y_pred))

9197.2735213773
0.5332348392412707
264456830.38296753


In [ ]:
LassoCV:
 R² : 0.5332
 MSE: 264456830.3830
 MAE: 9197.2735